In [14]:
import os
import vllm
from hidden_states import get_all_hidden_states
from vllm import LLM


In [15]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"

model_name = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"


In [16]:
llm = LLM(model=model_name, task="reward", tensor_parallel_size=1)

INFO 07-10 20:40:04 [config.py:1472] Using max model len 32768


2025-07-10 20:40:04,572	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-10 20:40:04 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-10 20:40:04 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250710) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidd

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.26s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.27s/it]


INFO 07-10 20:40:09 [default_loader.py:272] Loading weights took 2.37 seconds


INFO 07-10 20:40:09 [model_runner.py:1255] Model loading took 2.8876 GiB and 2.642731 seconds


In [17]:
slow_thinking=["Please give the detailed thinking progress.","Output the complete thinking steps."]
fast_thinking=["Output the answer directly.","Just give me the answer."]
texts = []
for t in slow_thinking+fast_thinking:
    texts.append(f"""<|im_start|>user
{t}<|im_end|>
<|im_start|>assistant
That""")
all_hidden_states, outputs = get_all_hidden_states(llm, texts)

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  6.86it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [18]:
from steer import (extract_statistical_control_vector,
    extract_diffmean_control_vector,
    extract_pca_control_vector,
    extract_lat_control_vector,
    extract_gemmascope_sae_diffmean_control_vector,
    extract_linear_probe_control_vector,
    StatisticalControlVector
)

In [19]:
control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,  # 三维列表 [样本][layer][token]
    positive_indices=[0, 1],     # 正样本索引
    negative_indices=[2, 3],     # 负样本索引
    model_type="qwen2.5",
    token_pos=-1,      # 使用最后一个token（默认）
    normalize=True
)
vector_name="temp/thinking_switch_pca_vector.gguf"


Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 792.55it/s]


In [20]:
control_vector.export_gguf(vector_name)
control_vector = StatisticalControlVector.import_gguf(vector_name)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([ 2.2973705e-07, -5.1668502e-04,  1.8600659e-02, ...,
         3.5134584e-02,  4.2884856e-02,  3.7976351e-02],
       shape=(1536,), dtype=float32), 1: memmap([-0.01999071,  0.03310987, -0.00749657, ..., -0.00187414,
        -0.01218193, -0.01811672], shape=(1536,), dtype=float32), 2: memmap([ 0.0041185 ,  0.0122271 ,  0.01879113, ..., -0.02213749,
        -0.02085043, -0.03191917], shape=(1536,), dtype=float32), 3: memmap([ 0.00154402, -0.01524673,  0.07874262, ..., -0.01505374,
        -0.00781636, -0.01621172], shape=(1536,), dtype=float32), 4: memmap([ 0.01440482, -0.01530505,  0.0639211 , ..., -0.00504166,
         0.02529835,  0.00720238], shape=(1536,), dtype=float32), 5: memmap([-0.01287344,  0.02896551, -0.0341954 , ..., -0.02252873,
        -0.01729885, -0.01689655], shape=(1536,), dtype=float32), 6: memmap([-0.03655712, -0.00266563, -0.01523219, ..., -0.05255105,
        -0.01028173,  0.028179